In [1]:
import cv2
import numpy as np
from robocup import localization as loc
from robocup import field
import math

In [ ]:
%timeit 

In [20]:
w = 500
img = cv2.imread('slike/9.jpg', 0)        

f = np.array(img, copy=True)
f2 = np.array(img, copy=True)
f3 = np.array(img, copy=True)
x, _, lines = loc.find_xy(f, w, 20, 0.2, 0.005)
side = 1
if loc.side_x(250, 250, lines[0][0], lines[0][1], f):
    side = -1
f2 = loc.draw_rect_line(f2, lines[0], side, 600*field.scale_field, 620*field.scale_field)
loc.draw_line_polar(f2, lines[0])
_, y, lines = loc.find_xy(f2, w, 20, 0.2, 0.005)
if y == -1:
    f3 = loc.draw_rect_line(f3, lines[0], side, 0, 620*field.scale_field)
    f3 = loc.draw_rect_line(f3, lines[0], side, (600+620)*field.scale_field, 800*field.scale_field)
    _, y, lines = loc.find_xy(f3, w, 20, 0.2, 0.005)
print(x, y)

    

            

f2 = loc.draw_cross(f2, 250, 250, 20, 255)   
cv2.imshow("2", f2)
#cv2.imshow("1", f)
#cv2.imshow("0", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

1618.3336747816556 1230.4588841224174


In [42]:
def find_xy(f, w, line_sens, lines_min_dist, right_angle_deviation):
    x_coor = -1
    y_coor = -1
    lines = cv2.HoughLines(f,2,np.pi/180*2,line_sens)

    if lines is not None:
        nms_lines = []
        print(len(lines), lines)
        found = False
        for i in range (len(lines)):
            l = lines[i]
            print(l)
            l = l[0]
            if True:
                rho = l[0]
                theta = l[1]
                print(rho, theta, "l")
                
                if loc.lines_d (l, nms_lines, w*1.41, math.pi) > lines_min_dist:
                    if True: #not found or loc.is_on_right(nms_lines[0], l, right_angle_deviation):
                        nms_lines.append(l)
                        found = True
                        f = loc.draw_line_polar(f, l)
                        print("l= ", l)


                        dist = loc.pl_dist_polar2(w/2, w/2, rho, theta)
                        dist2 = dist / field.scale_field
                        if loc.is_y(l) and x_coor == -1:
                            if loc.side_x(int(w/2), int(w/2), rho, theta, f):
                                x_coor = field.x - dist2
                                print("l1= ", l)
                            else:
                                x_coor = dist2
                                print("l2= ", l)
                        elif not loc.is_y(l) and y_coor == -1:   
                            if not loc.side_y(int(w/2), int(w/2), rho, theta, f):
                                y_coor = field.y - dist2
                                print("l3= ", l)
                            else:
                                y_coor = dist2
                                print("l4= ", l)
                        
                        if x_coor!= -1 and y_coor != -1:
                            return x_coor, y_coor, nms_lines, f
                        
                        print("x = ", x_coor, "y = ", y_coor, " dist= ", dist, " dist2 = ", dist2)#, end = '\r') #, " rho=", rho, " theta=", theta, " a=", a, " b=", b, " pres=", x0, " rho=", rho, " theta = ", theta, " end " , end='\r')
    return x_coor, y_coor, nms_lines, f